### Notebook for training used on the provided dataset


Using the NVIDIA architecture
This is mainly used for looking at the potential preprocessing steps, model architecture etc.

In [1]:
import os
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
from scipy import ndimage
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, Cropping2D
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
# import cv2

Using TensorFlow backend.


In [2]:
path_label = "data/driving_log.csv"
path_img = "data/IMG"

In [3]:
def get_labels(path_label="data/driving_log.csv"):
    df = pd.read_csv(path_label)
    labels = df["steering"].values
    print("Got labels")
    return labels

In [4]:
def get_images(path_img="data/IMG"):
    images = []
    fnames = os.listdir(path_img)
    for fname in fnames:
        # only use CENTER images first
        if fname.startswith("center"):
            img = plt.imread(os.path.join(path_img, fname))
            images.append(img)
    images = np.array(images)
    print("Got images")
    return images

In [5]:
def build_model():
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))  # normalize
    # add cropping
    model.add(Cropping2D(cropping=((70, 25), (0, 0))))
    model.add(Conv2D(24,(5,5), strides=(2,2), activation="relu"))
    model.add(Conv2D(36, (5,5), strides=(2,2), activation="relu"))
    model.add(Conv2D(48, (5,5), strides=(2,2), activation="relu"))
    model.add(Conv2D(64, (3,3), strides=(1,1), activation="relu"))
    model.add(Conv2D(64, (3,3), strides=(1,1), activation="relu"))
    model.add(Flatten())
    model.add(Dense(1164))
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    return model

In [6]:
y_train = get_labels()
X_train = get_images()

Got labels
Got images


In [8]:
model = build_model()

In [9]:
model.compile(loss=keras.losses.mse,
              optimizer=keras.optimizers.Adam())

In [ ]:
model.fit(X_train, y_train, validation_split=0.2, batch_size=128, epochs=5, shuffle=True)

Train on 6428 samples, validate on 1608 samples
Epoch 1/5


In [ ]:
# model.save("model.h5")
model.save("model.h5")
print("model saved")

In [ ]:
K.clear_session()

In [ ]:
# ToDos for first step

# write generator for iterative loading of images
# add regularization stuff (batch_norm, dropout)
# train and save model
# deploy model and test

#### Further Experimentations:


- different processing strategies in lambda layer
- Image augmentation (flipping)
- cropping images (top and bottom)
- use left and right images for recovery
- modify model architecture
- iterative, "transfer" learning approach (see https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/Behavioral+Cloning+Cheatsheet+-+CarND.pdf)
